In [1]:
from sources.api.bus import *
from sources.api.train import *
from sources.api.yahoo_transit import *

/Users/x0y14/dev/reachable-area/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
kanagawa_stops = load_stop_data("../dataset/busstpos/kanagawa/P11-22_14.geojson")
stations = load_station_data("../dataset/stations/N02-20_Station.geojson")

In [3]:
# 検索時をイメージして、入力から駅データを取り出す。
FRONT_OF_KAIT_STOP = None  # 神奈川工科大学前
INSIDE_OF_KAIT_STOP = None  # 神奈川工科大学(校内)

for stop in kanagawa_stops:
    if "神奈川工科大学前" == stop.name:
        FRONT_OF_KAIT_STOP = stop
    elif "神奈川工科大学" == stop.name:
        INSIDE_OF_KAIT_STOP = stop

print(str(FRONT_OF_KAIT_STOP), str(INSIDE_OF_KAIT_STOP))

神奈川中央交通（株）-神奈川工科大学前 神奈川中央交通（株）-神奈川工科大学


In [4]:
# バス路線の取得
kait_routes = list(set(FRONT_OF_KAIT_STOP.routes + INSIDE_OF_KAIT_STOP.routes))
print(kait_routes)

# バス路線上のバス停の取得
stops_of_kait_routes = []
for stop in kanagawa_stops:
    for kait_route in kait_routes:
        if (kait_route in stop.routes) and (stop not in stops_of_kait_routes):
            stops_of_kait_routes.append(stop)

#全てのバス会社から検索するのは時間がかかるので、同じバス会社に限る
BUS_GROUP = FRONT_OF_KAIT_STOP.group

['厚67', '厚07', '厚89']


In [5]:
# 350円で行ける場所
# 1. バスのみ

rule_max_fare = 350
rule_max_transfer = 0

stops_fare_350 = []

for kait_stop in [FRONT_OF_KAIT_STOP, INSIDE_OF_KAIT_STOP]:
    for stop in kanagawa_stops:
        if BUS_GROUP != stop.group:
            continue
        routes = get_route_yahoo_transit(
            TransitType.BUS,
            generate_bus_stop_name(stop),
            generate_bus_stop_name(kait_stop))
        for route in routes:
            if (route["fare"] <= rule_max_fare) and (route["transfer"] <= rule_max_transfer) and (
                    stop not in stops_fare_350):
                stops_fare_350.append(stop)

for stop in stops_fare_350:
    print(str(stop))

神奈川中央交通（株）-三本松
神奈川中央交通（株）-新中野橋
神奈川中央交通（株）-亀井
神奈川中央交通（株）-亀井橋
神奈川中央交通（株）-第一住宅
神奈川中央交通（株）-小田急住宅前
神奈川中央交通（株）-下川入
神奈川中央交通（株）-才戸橋
神奈川中央交通（株）-あつぎ郷土博物館
神奈川中央交通（株）-棚沢
神奈川中央交通（株）-鳶尾一丁目
神奈川中央交通（株）-鳶尾団地
神奈川中央交通（株）-鳶尾小学校入口
神奈川中央交通（株）-西四ツ谷
神奈川中央交通（株）-鳶尾団地東
神奈川中央交通（株）-リコー前
神奈川中央交通（株）-子中
神奈川中央交通（株）-荻野新宿
神奈川中央交通（株）-宿原入口
神奈川中央交通（株）-糀屋前
神奈川中央交通（株）-松蓮寺
神奈川中央交通（株）-妻田薬師
神奈川中央交通（株）-中村入口
神奈川中央交通（株）-妻田
神奈川中央交通（株）-三家入口
神奈川中央交通（株）-木売場
神奈川中央交通（株）-市立病院前
神奈川中央交通（株）-税務署入口
神奈川中央交通（株）-厚木東町
神奈川中央交通（株）-厚木ナイロン
神奈川中央交通（株）-市役所入口
神奈川中央交通（株）-あつぎ大通り
神奈川中央交通（株）-厚木バスセンター
神奈川中央交通（株）-本厚木駅
神奈川中央交通（株）-本厚木駅南口
神奈川中央交通（株）-鶴嶺八幡宮
神奈川中央交通（株）-入ノ田
神奈川中央交通（株）-新宿


In [ ]:
# 2.バスと